In [2]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [3]:
llm.invoke('hi')

'Hello! How can I assist you today? 😊 If you have any questions or need information, feel free to let me know.'

In [4]:
llm.invoke('سلام. خوبی؟')

'سلام! من اگر بتوانم کمکت را در پاسخ به سوالات و مشورت\u200cهای خودت انجام دهم، خوشحالم. چطوری هستی؟ آیا کاری داری که باید برایم توضیح دهی یا سوالی پرسیده\u200cای؟'